# A Deep Learning-assisted Template Attack Against Dynamic Frequency Scaling Countermeasures

In [5]:
import numpy as np
import h5py

from CNN.train import train
from CNN.prepare_dataset import createDataset
from utils.segmentation import getModule, segmentAlignGradCAM, segmentAlignGT
from templateAttack import TemplateAttack

## CNN dataset creation

In [ ]:
DFS_DESYNCH_path = "</path/to/DFS_DESYNCH/folder>"
dataset_out_path = "</path/to/output/folder>"

In [ ]:
createDataset(DFS_DESYNCH_path, dataset_out_path)

## CNN training

Each CNN is configure thanks to a YALM configuration file.  
You can set different module hyper-parameters as well as the dataset, the logger, and the experiment configurations.  
Default configuration are in `CNN/configs` directory, both for Neputune logger and for the experiment. 

In [ ]:
config_folder = "CNN/configs/exp_v1/" # /path/to/experiment/config/folder/
train(config_folder)

## Segmentation

In [ ]:
output_path = "</path/to/output_file>"
DFS_DESYNCH_path = "</path/to/DFS_DESYNCH>"
SID = "<Neptune_SID>"

Firt, get the trained CNN starting from Neptune SID.  
Then, align attack traces to reference frequency exploiting Grad-CAM.

In [ ]:
module = getModule(SID)
with h5py.File(DFS_DESYNCH_path, 'r') as dfs_desynch:
    traces = dfs_desynch['attack/traces']
    segmentAlignGradCAM(traces, module, output_path, force=True)

## Template Attack

In [ ]:
profiling_aligned_path = "</path/to/aligned/output/path>"
DFS_DESYNCH_path = "</path/to/DFS_DESYNCH>"
template_path = "</path/to/save/template>"

Align profiling traces to a reference frequency.

In [ ]:
with h5py.File(DFS_DESYNCH_path, 'r') as dfs_desynch:
    traces = dfs_desynch['profiling/traces']
    frequencies = dfs_desynch['profiling/labels']
            
    segmentAlignGT(traces, frequencies, profiling_aligned_path, force=True)

Create a template for attacking.

In [ ]:
traces = np.load(profiling_aligned_path, mmap_mode='r')
with h5py.File(DFS_DESYNCH_path, 'r') as dfs_desynch:
    keys = dfs_desynch['profiling/metadata']['key']
    plaintexts = dfs_desynch['profiling/metadata']['plaintext']

ta = TemplateAttack(template_path, aggregate_n_samples=100)
ta.computePCA(traces, plaintexts, keys)
ta.fit(traces, plaintexts, keys)

Load attack metadata and traces already aligned through Grad-CAM.

In [ ]:
attack_aligned_file = "</path/to/aligned/output/file>"

traces = np.load(attack_aligned_file, mmap_mode='r')
with h5py.File(DFS_DESYNCH_path, 'r') as dfs_desynch:
    keys = dfs_desynch['attack/metadata']['key']
    plaintexts = dfs_desynch['attack/metadata']['plaintext']

In [ ]:
ge, gd, ranks = ta.guessingMetrics(traces, plaintexts, keys)
print(f"Guessing distance: {gd}, Guessing entropy: {ge}")